In [9]:
import os
import glob
import pandas as pd

# Directory path
directory = '/home/matias/Documents/media_monitor/data/rss_slices/'

# Find all CSV files in the directory
file_pattern = os.path.join(directory, '*.csv')
csv_files = glob.glob(file_pattern)

print(f"Found {len(csv_files)} CSV files.")


Found 122 CSV files.


In [10]:
# List to hold DataFrames
dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(dfs, ignore_index=True)


In [11]:
# Drop duplicates on all columns
deduped_df = combined_df.drop_duplicates()

# OR drop duplicates based on specific columns
deduped_df = combined_df.drop_duplicates(subset=['Title', 'Source'])


In [12]:
deduped_df.shape

(6032, 7)

In [13]:
deduped_df['day'] = pd.to_datetime(deduped_df['Published'], format='mixed').dt.strftime('%Y-%m-%d')

deduped_df.groupby('day').size().tail(30)

/tmp/ipykernel_3230405/788144331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deduped_df['day'] = pd.to_datetime(deduped_df['Published'], format='mixed').dt.strftime('%Y-%m-%d')


day
2025-05-12      2
2025-05-13      1
2025-05-14      7
2025-05-15      3
2025-05-16      2
2025-05-18      4
2025-05-19      1
2025-05-20      6
2025-05-21      4
2025-05-22      1
2025-05-23      9
2025-05-24      9
2025-05-25     18
2025-05-26     92
2025-05-27     94
2025-05-28    246
2025-05-29    452
2025-05-30    473
2025-05-31    258
2025-06-01    222
2025-06-02    466
2025-06-03    496
2025-06-04    544
2025-06-05    540
2025-06-06    490
2025-06-07    276
2025-06-08    199
2025-06-09    474
2025-06-10    459
2025-06-11     91
dtype: int64

In [14]:
links = deduped_df.sort_values('day').tail(20)['Link'].values
links = list(links)


In [15]:
import os
import time
import json
import pyperclip
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# links = [
#     "https://example.com/page1",
#     "https://example.com/page2",
#     # ...
# ]

output_file = 'scraped_links.jsonl'
sleep_time = 5  # seconds
page_timeout = 15  # seconds

scraped_results = []

for idx, url in enumerate(tqdm(links)):
    options = Options()
    options.add_argument("--start-maximized")
    # options.add_argument("--headless")  # Uncomment to run without UI

    driver = None
    try:
        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(page_timeout)

        driver.get(url)
        time.sleep(sleep_time)  # Let dynamic content load

        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.CONTROL, 'a')
        body.send_keys(Keys.CONTROL, 'c')
        time.sleep(1)  # Clipboard delay

        scraped_html = pyperclip.paste()

        scraped_results.append({
            'index': idx,
            'url': url,
            'scraped_data': scraped_html
        })
        print(f"✅ Fetched data from: {url}")

    except Exception as e:
        print(f"⚠️ Error scraping URL: {url}\n{e}")

    finally:
        if driver:
            driver.quit()

# Save results
with open(output_file, 'w', encoding='utf-8') as f:
    for record in scraped_results:
        json.dump(record, f, ensure_ascii=False)
        f.write('\n')

print(f"Done! {len(scraped_results)} pages saved to {output_file}.")


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Fetched data from: https://news.google.com/rss/articles/CBMiqAFBVV95cUxPT0IxazktYVV0elU3b25RVzRrdm1oeDBnVDZja1F6VmF2SDF1WFIyRW9BaGtwOFZrMFNMSnJPWmNmUWJScjhhZ3hJbTBNR0s5UjNWVGpfOXpSdTliU1c1WExUdEpxSTcyaHg4QWZpQnY5SDloUG53TDN6RFNXbkt6MXNmUUhNVzVxSkhzVXBXOHlfV3g1VC1FRXZiYTd2OXh1M0wySTIwZjHSAcMBQVVfeXFMTkNOd0xpU0xORV82ZzBqSjl6Q2RsWkl6VEV5Q3dZZGhkZnM1UGZOTkNfdGcxOGlDOTJfbkhzc3g3b29zVTBDUjA2bkpZanZ1SXZFeVVYZjFVUVU1a3ZyUERTZ1JZd01aaUduaTh4S0NLcHVJamtMeUxIblEybWh2QmV4Zml3QjU4SWdTR2xrYjczcTl5RlQ5Y3FJRE00bUFYWmFPcEZwcjZiMlhkck9PeWVtSGtjaHJaVWprTFVadWdUX3Yw?oc=5


  5%|▌         | 1/20 [00:20<06:29, 20.51s/it]

✅ Fetched data from: https://news.google.com/rss/articles/CBMirgFBVV95cUxQaFc0R2xqdFk2QzNNSkJMcXRuTEdGWUY3ZVZoYXNxY243aVJGTFpVVE4xMFpWZWtHY3hGSkxrbF9IVzlkRjJUdVZucU8xWHhqelZwYW5iYXlOTjlhRVpEZFIxVXNCT3EyanJueTRCc0ozMzF1N3RTQ213THp5WTdac1c2bjNnV2xEeVppQUFPQWRyRmc1VGFrbTFEblVpY3dfNWZ0NnVHYTRNZVlEeEHSAcgBQVVfeXFMUHI0WndldXFMV2pQb2VJSzJzS2VORTRfMmg1RVh2M1lHUmxZUTUyNHpEcDlTZFUxcV9mU1ktZ0pJNUVpMUZ2ZnVyUjN1eGVsVm56amN2V3daVUNTaWpJbm93M3NNcUczeWE1T1k2bkFQcVI0WDEwcy1selFkV0RTN2lxcnp5RkNQM0tkZDlEQW1WNlYzM3VIX2xfcWNfREp5OGxOWXM4b3hPdk1CRk5LM0dFMk5yQnRtR2RSOGJ0Nzd5a2daSmlVQmo?oc=5


 10%|█         | 2/20 [00:41<06:13, 20.78s/it]

...

✅ Fetched data from: https://news.google.com/rss/articles/CBMi8gFBVV95cUxONUgtU2ZVRldPTThfSG03UmdmdUhFbzkzVF9Ha2p2NW83QlFaaVpvNU9nVktHeFZ0NndnS1YzdFZjSDBPNExBRWZJNjhDR05NbmNjU1VKeGZFRER4RUhRamp2SGhMTlVwNzl5S0RKaldmYWZMZWRESDZTdmNUZUVFS19pcGdLM01FQmJsMWhWTWdKWEoxR2V4VXRYVjQ3aWJXRlN0cnJ3ZmlJVzVYZm1la3hlRmQ0MXBGYmx5SE9nMTRKdDlBUHUweV9rZEhITU5xeHF2NUZHT000dGc1WkZLRFp3OVZVQzduZW5hMm1QeDd5UQ?oc=5


100%|██████████| 20/20 [07:09<00:00, 17.16s/it]

✅ Fetched data from: https://news.google.com/rss/articles/CBMi0gFBVV95cUxQN2FhQWJLaVEyZmM4WTlSRlpNMnU5Wk1HajZYeU51TjlUUHhLdnkybkxLa1p0b2RmYzl6OGZRZ0wwUjZmb2pVOGFSOGoxZVB5Q2ZoZEd0aFZZVGhHbnlhNHp0YjJydVhvRndFUi1aZW1QWno3NFI3MU4zZVFmc3NrYnFNNkZPMW1PMUZwU0lxNkhfMzE2Um5TMGR2OGtqeEgyYnpNNmJGUXktZ1FYd29OamtVdk91QkxxVzQxdlhrVE40OEtZdW10Y2tBV0dNWHltT2fSAdcBQVVfeXFMTXVUc3BsQUZJSkxyaTloTmtDMkktRHE0bGhQRDA0eU9rakF4WTlmaGhZUDZyWk10am1OWUQ1bVBVZ0JIU205N1g0WE0xN0VZWUQxenptU3NEdVplUWI2X0ltOFEtZGNpU3pSSlJYUy1vdERRYTZoWnBPZUZZYmF4d0ZQQ0pDQnJXZUVPSXFkZnFwVnRRMmdNZkFHcE96OUt5M3EtOFBjelVwUk5ld0hHRFM3S0QtbWE0a0RrVDVZcGx1Y2lhbkd5Q2xUVzQ0ZWVCY1c4TWh1Yk0?oc=5


100%|██████████| 20/20 [07:09<00:00, 21.49s/it]

Done! 18 pages saved to scraped_links.jsonl.
